In [1]:
from __future__ import division
import numpy as np

# import MalmoPython
import malmo.MalmoPython as MalmoPython
import os
import random
import sys
import time
import json
import random
import math
import errno
from builtins import range
from past.utils import old_div
from collections import defaultdict, deque
from timeit import default_timer as timer
from pprint import pprint

# RL Agent Object

In [ ]:
actions_space = ["movenorth 1", "movesouth 1", "movewest 1", "moveeast 1", "jumpnorth 1", "movesprint 1"]

DIAMOND_WALL_Z = 25
INITIAL_DISTANCE = DIAMOND_WALL_Z - (-3)
state_actions =  {
            'diamond_block': 100,
            'packed_ice': 1,
            'log': -10,
            'water': -5 
            }
class Racer(object):
    def __init__(self, alpha=0.3, gamma=1, n=1):
        """Constructing an RL agent.

        Args
            alpha:  <float>  learning rate      (default = 0.3)
            gamma:  <float>  value decay rate   (default = 1)
            n:      <int>    number of back steps to update (default = 1)
        """
        self.epsilon = 0.2  # chance of taking a random action instead of the best
        self.q_table = {}
        self.n, self.alpha, self.gamma = n, alpha, gamma
        self.actions_taken = defaultdict(lambda: 0, {})
        self.num_actions = 0
        self.currentState = {
            'agent_x': 1,
            'agent_z': -4,
            'goal_z': 25,
            'has_water_left': False,
            'has_water_right': False,
            'has_water_forward': False,
            'has_wood_left': False,
            'has_wood_right': False,
            'has_wood_forward': False,
        }

    def clear_actions(self):
        """Resets the actions in case of a new iteration to fetch. """
        self.actions_taken = defaultdict(lambda: 0, {})
        self.num_actions = 0
        self.currentState = {
            'agent_x': 1,
            'agent_z': -4,
            'goal_z': 25,
            'has_water_left': False,
            'has_water_right': False,
            'has_water_forward': False,
            'has_wood_left': False,
            'has_wood_right': False,
            'has_wood_forward': False,
        }
        #resetDefault
        
    @staticmethod
    def get_curr_location():
        # get the world state
        world_state = agent_host.peekWorldState()
        location = tuple()
        initial_location = (1, 2, -3)
#         print("get curr location")
        if world_state.number_of_observations_since_last_state > 0:
            msg = world_state.observations[-1].text
#             print(msg)
            observations = json.loads(msg)
#             print(observations)

            # get curr location from json
            xpos = observations.get(u'XPos',0)
            ypos = observations.get(u'YPos',0)
            zpos = observations.get(u'ZPos',0)
            location = (xpos, ypos, zpos)
            print("LOCATION: ", location)
            return location
        else:
            return initial_location
    
    
        
    def calculate_reward(self):
        agent_z = Racer.get_curr_location()[2]
        dist_from_wall = DIAMOND_WALL_Z - agent_z
        dist_traveled = INITIAL_DISTANCE - dist_from_wall
        reward = math.floor((dist_traveled*100)/INITIAL_DISTANCE)
        return reward
        
    def present_actions(self, agent_host):
        """Calculates the reward points for the current inventory.

        Args
            agent_host: the host object

        Returns
            reward:     <float> current reward from world state
        """
        current_r = 0
        #time.sleep(0.1)
        
        # calculate agent's current location
        # measure the distance between current location and the diamond wall
        

#         for item, counts in self.actions_taken.items():
#             current_r += rewards_map[item] * counts
#             current_r += rewards_map[item]
        current_r = self.calculate_reward()

        #agent_host.sendCommand('quit')
        #time.sleep(0.25)
        return current_r
    
    def is_solution(reward):
        """If the reward equals to the maximum reward possible returns True, False otherwise. """
        return reward == 100
        # THIS VALUE IS SUBJECT TO CHANGE
    
    # do we need the below function?
    def get_possible_actions(self, agent_host, is_first_action=False):
        """Returns all possible actions that can be done at the current state. """
#         print("in get possible actions")
        action_list = []
#         if not is_first_action:
#             # Not allowing Odie to come back empty.
# #             action_list = ['present_gift']
#             action_list = ['present_actions']
        action_list.extend(actions_space)
        return action_list

    def get_curr_state(self): #NEEDS WORK
        """Creates a unique identifier for a state.
        Probably want to see the 3 blocks in front of it as well. Like if there is an obstacle. Check Z+1
        """
        state = self.get_curr_location()
        return state
    
    def choose_action(self, curr_state, possible_actions, eps):
        """Chooses an action according to eps-greedy policy. """
#         print("in choose action")
        if curr_state not in self.q_table:
            self.q_table[curr_state] = {}
        for action in possible_actions:
            if action not in self.q_table[curr_state]:
                self.q_table[curr_state][action] = 0
        
        rnd = random.random()
        if rnd <= eps:
            a = random.randint(0, len(possible_actions) - 1)
            return possible_actions[a]
        else:
            # copy dict{actions: q-values} of q_table[curr_state]
            state_actions =self.q_table[curr_state]
            # find the max q-value
            max_q = max(state_actions.values())
            # find the list of actions that return the maximum q-value
            max_actions = [action for action, value in state_actions.items() if value == max_q]
            # pick a random action from the max_actions list
            max_rand = random.randint(0, len(max_actions) - 1)
            return max_actions[max_rand]
                     
    def act(self, agent_host, action):
        print(action + ",", end = " ")
        agent_host.sendCommand(action)
        return 0

    def update_q_table(self, tau, S, A, R, T): # THIS OR THE COPY OF IT NEEDS WORK!
        """Performs relevant updates for state tau.

        Args
            tau: <int>  state index to update
            S:   <dequqe>   states queue
            A:   <dequqe>   actions queue
            R:   <dequqe>   rewards queue
            T:   <int>      terminating state index
            
        """
        curr_s, curr_a, curr_r = S.popleft(), A.popleft(), R.popleft()
        G = sum([self.gamma ** i * R[i] for i in range(len(S))])
        if tau + self.n < T:
            G += self.gamma ** self.n * self.q_table[S[-1]][A[-1]]

        old_q = self.q_table[curr_s][curr_a]
        self.q_table[curr_s][curr_a] = old_q + self.alpha * (G - old_q)
        
        ###Incomplete version of Q Table. Figuring out best way to handle. TD seems to be a better algo.
    def update_q_table2(self, S, A, R, T):
        """Performs relevant updates for the Q-values.
        TD(0) implementation of SARSA algorithm. Better for this use case.
        Args
            S:   <dequqe>   states queue
            A:   <dequqe>   actions queue
            R:   <dequqe>   rewards queue
            T:   <int>      terminating state index

        """
        G = 0
        for i in range(len(S)-1, -1, -1):
            G = self.gamma * G + R[i+1]
            old_q = self.q_table[S[i]][A[i]]
            self.q_table[S[i]][A[i]] = old_q + self.alpha * (G - old_q)

    def best_policy(self, agent_host):
        """Reconstructs the best action list according to the greedy policy. """
        self.clear_actions()
        policy = []
        current_r = 0
        is_first_action = True
        next_a = ""
        while next_a != "present_actions":
            curr_state = self.get_curr_state()
            possible_actions = self.get_possible_actions(agent_host, is_first_action)
            next_a = self.choose_action(curr_state, possible_actions, 0)
            policy.append(next_a)
            is_first_action = False
            current_r = self.act(agent_host, next_a)
        print(' with reward %.1f' % (current_r))
        return is_solution(current_r)
        #print 'Best policy so far is %s with reward %.1f' % (policy, current_r)
        
    def run(self, agent_host): ###THIS NEEDS WORK ALONG W THE Q TABLE!
        """Learns the process to reach the diamonds"""
        S, A, R = deque(), deque(), deque()
        present_reward = 0
        done_update = False
        while not done_update:
            s0 = self.get_curr_state()
            possible_actions = self.get_possible_actions(agent_host, True)
            a0 = self.choose_action(s0, possible_actions, self.epsilon)
            S.append(s0)
            A.append(a0)
            R.append(0)

            T = sys.maxsize
            for t in range(sys.maxsize):
                time.sleep(0.1)
                if t < T:
                    print("action: ", A[-1])
                    
                    time.sleep(1)
                    
                    current_r = self.act(agent_host, A[-1])
                    R.append(current_r)

                    if A[-1] == "present_actions": #We do not needs this.
                        # Terminating state
                        T = t + 1
                        S.append('Term State')
                        present_reward = current_r
                        print("Reward:", present_reward)
                    else:
                        s = self.get_curr_state()
                        S.append(s)
                        possible_actions = self.get_possible_actions(agent_host)
                        next_a = self.choose_action(s, possible_actions, self.epsilon)
                        A.append(next_a)

                tau = t - self.n + 1
                if tau >= 0:
                    self.update_q_table(tau, S, A, R, T)

                if tau == T - 1:
                    while len(S) > 1:
                        tau = tau + 1
                        self.update_q_table(tau, S, A, R, T)
                    done_update = True
                    break
                    
            

# RL Racer Object

In [7]:
rewards_map =  {'diamond_block': 100, 'packed_ice': 1, 'log': -10, 'water': -5}
actions_space = ["move 1", "move -1", "strafe 1", "strafe -1"]  
# REMINDER: VALUE OF EACH ACTION IS THE SPEED, NOT NUMBER OF TIMES
# move  1    full speed ahead
# move -1    full speed backwards
# strafe 1   moves right at full speed
# strafe -1  moves left at full speed
# turn 1     turns full speed right
# turn -1    turns full speed left
# jump 1/0   starts/stops jumping



# helper function to update the actions of the current state
def get_curr_state_helper(items):
    curr_state_items = tuple()
    for key, val in items:
        curr_state_items += (key, val)
    
    return curr_state_items


# helper function to choose an action based on the current state and possible actions
def choose_action_helper(curr_state, possible_actions, eps, q_table):
    rnd = random.random()
    
    action_chosed = ""
    if rnd < eps:
#         print("random choice")
        a = random.randint(0, len(possible_actions) - 1)
        action_chosed = possible_actions[a]
    else:
#         print("greedy choice")
        d = q_table[curr_state]  # create dict of q_table with (action, qValue) pairs
        max_qValue = max(d.values())  # extract maximum Q-value
        
        ls = [k for k, v in d.items() if v == max_qValue]  # get all actions with same maximum Q-value
#         print("ls: ", ls)

        # randomly choose from list of possible actions with same maximum Q-value
        a = random.randint(0, len(ls) - 1)  
        action_chosed = ls[a]
    
#     print("action chosed: ", action_chosed)
    
    return action_chosed
            

class Racer(object):
    # initializing the parameters for the agent
    def __init__(self, alpha=0.3, gamma=1, n=1):
        """Constructing an RL agent.

        Args
            alpha:  <float>  learning rate      (default = 0.3)
            gamma:  <float>  value decay rate   (default = 1)
            n:      <int>    number of back steps to update (default = 1)
        """
        self.epsilon = 0.4  # chance of taking a random action instead of the best, default 0.2
        self.q_table = {}   # track agent decisions and rewards
        self.n, self.alpha, self.gamma = n, alpha, gamma
        self.actions_taken = defaultdict(lambda: 0, {})
        self.num_actions = 0
        
    def clear_actions(self):
        """Resets the actions taken in case of a new attempt to fetch."""
        self.actions_taken = defaultdict(lambda: 0, {})
        self.num_actions = 0
    
    # where the agent will present the final list of actions taken 
    def present_result(self, agent_host):
        """Calculates the reward points for the current list of actions taken.

        Args
            agent_host: the host object

        Returns
            reward:     <float> current reward from world state
        """
        current_r = 0
        # time.sleep(0.1)

        for item, counts in self.actions_taken.items():
            current_r += rewards_map[item] * counts

        agent_host.sendCommand('quit')
        #time.sleep(0.25)
        return current_r
    
    @staticmethod
    def is_solution(reward):
        """If the reward equals to the maximum reward possible returns True, False otherwise. """
        return reward == 0  # default 0
    
    @staticmethod
    def get_obj_locations(agent_host):
        """Queries for the object's location in the world.

        As a side effect it also returns the Racer's location.
        """
        objs = {}
        while True:
            world_state = agent_host.getWorldState()
            if world_state.number_of_observations_since_last_state > 0:
                # print(world_state.observations)
                msg = world_state.observations[-1].text
                data = json.loads(msg)
                # print("data:", data)
                # print("data.keys():", data.keys())

                for obj in data['floorAll']:
                    # print(obj, '\n')
                    
                    # need to work on finding specific coordinates for each object
                    # prob can just extract coordinates of each entity from xml
                    if obj in objs:
                        objs[obj] += 1
                    else:
                        objs[obj] = 0
                
                objs['Racer'] = (data['Yaw'], data['XPos'], data['ZPos'])
                print("objs:", objs)
                return objs
    
    
    # heavy emphasis on this function
    # WORK ON THIS
    def get_possible_actions(self, agent_host, is_first_action=False):
        """Returns all possible actions that can be done at the current state. """
        action_list = []
        if not is_first_action:
            # Not allowing the Racer to come back empty.
            action_list = ['present_result']
            
        nearby_obj = self.get_obj_locations(agent_host)
        # print("nearby_obj", nearby_obj)
        if len(nearby_obj) > 1:
            action_list.extend([item for item in nearby_obj.keys() if item != 'Racer'])
            
        return action_list
    
    
    
    
    def get_curr_state(self):
        """Creates a unique identifier for a state.

        The state is defined as the items in the agent's list of choices. Notice that the state has to be sorted -- otherwise
        differnt order in the list will be different states.
        """
        return get_curr_state_helper(self.actions_taken.items())
    
    def choose_action(self, curr_state, possible_actions, eps):
        """Chooses an action according to eps-greedy policy. """
        if curr_state not in self.q_table:
            self.q_table[curr_state] = {}
        for action in possible_actions:
            if action not in self.q_table[curr_state]:
                self.q_table[curr_state][action] = 0
        
        return choose_action_helper(curr_state, possible_actions, eps, self.q_table)
    
    
    
    # heavy emphasis on this function
    # completes the action to either move in a certain direction or jump 
    # WORK ON THIS
    def complete_action(self, agent_host, action):
        print("action completed:", action)
        agent_host.sendCommand(action)
        
        self.actions_taken[action] += 1
        self.num_actions += 1
        time.sleep(1)
        
        
        
    # act function determines if agent should present its results or do another action
    def act(self, agent_host, action):
        print(action + ",", end = " ")
        if action == 'present_result':
            return self.present_result(agent_host)
        else:
            self.complete_action(agent_host, action)
        return 0
    
    def update_q_table(self, tau, S, A, R, T):
        """Performs relevant updates for state tau.
        Args
            tau: <int>  state index to update
            S:   <dequqe>   states queue
            A:   <dequqe>   actions queue
            R:   <dequqe>   rewards queue
            T:   <int>      terminating state index
        """
        curr_s, curr_a, curr_r = S.popleft(), A.popleft(), R.popleft()
        G = sum([self.gamma ** i * R[i] for i in range(len(S))])
        if tau + self.n < T:
            G += self.gamma ** self.n * self.q_table[S[-1]][A[-1]]

        old_q = self.q_table[curr_s][curr_a]
        self.q_table[curr_s][curr_a] = old_q + self.alpha * (G - old_q)
    
    def best_policy(self, agent_host):
        """Reconstructs the best action list according to the greedy policy."""
        self.clear_actions()
        policy = []
        current_r = 0
        is_first_action = True
        next_a = ""
        
        while next_a != "present_result":
            curr_state = self.get_curr_state()
            possible_actions = self.get_possible_actions(agent_host, is_first_action)
            next_a = self.choose_action(curr_state, possible_actions, 0)
            policy.append(next_a)
            is_first_action = False
            current_r = self.act(agent_host, next_a)
        print(' with reward %.1f' % (current_r))
        #print 'Best policy so far is %s with reward %.1f' % (policy, current_r)
        
        return self.is_solution(current_r)
    
    def run(self, agent_host):
        """Learns the process to compile the best result for reaching the goal."""     
        S, A, R = deque(), deque(), deque()
        present_reward = 0
        done_update = False
        
        while not done_update:
            s0 = self.get_curr_state()
            possible_actions = self.get_possible_actions(agent_host, True)
            a0 = self.choose_action(s0, possible_actions, self.epsilon)
            S.append(s0)
            A.append(a0)
            R.append(0)
            
            T = sys.maxsize
            for t in range(sys.maxsize):
                time.sleep(0.1)
                if t < T:
                    current_r = self.act(agent_host, A[-1])
                    R.append(current_r)
                    
                    if A[-1] == "present_result":
                        # Terminating state
                        T = t + 1
                        S.append('Term State')
                        present_reward = current_r
                        print("Reward:", present_reward)
                    else:
                        s = self.get_curr_state()
                        S.append(s)
                        possible_actions = self.get_possible_actions(agent_host)
                        next_a = self.choose_action(s, possible_actions, self.epsilon)
                        A.append(next_a)
                        
                    tau = t - self.n + 1
                    if tau >= 0:
                        self.update_q_table(tau, S, A, R, T)

                    if tau == T - 1:
                        while len(S) > 1:
                            tau = tau + 1
                            self.update_q_table(tau, S, A, R, T)
                        done_update = True
                        break

# Run 

In [ ]:
if sys.version_info[0] == 2:
    sys.stdout = os.fdopen(sys.stdout.fileno(), 'w', 0)  # flush print output immediately
else:
    import functools
    print = functools.partial(print, flush=True)
    

expected_reward = 3390
my_client_pool = MalmoPython.ClientPool()
my_client_pool.add(MalmoPython.ClientInfo("127.0.0.1", 10000))


# Create default Malmo objects:
agent_host = MalmoPython.AgentHost()
try:
    agent_host.parse( sys.argv )
except RuntimeError as e:
    print('ERROR:',e)
    print(agent_host.getUsage())
    exit(1)
#     sys.exit()
if agent_host.receivedArgument("help"):
    print(agent_host.getUsage())
    exit(0)
#     sys.exit()


# load in world map
world_num = random.randint(0, 24)
# mission_file = 'xmls/world_{world_num}.txt'.format(world_num = world_num)
mission_file = 'xmls/testmap.txt'  # test map 
with open(mission_file, 'r') as f:
    print("Loading mission from %s" % mission_file)
    missionXML = f.read()

    
# initiate the Racer object
num_reps = 1
n = 10
racer = Racer(n=n)
print("n =", n)
racer.clear_actions()


for iRepeat in range(num_reps):
    my_mission = MalmoPython.MissionSpec(missionXML, True)
    my_mission_record = MalmoPython.MissionRecordSpec()  # Records nothing by default
    my_mission.requestVideo(1260, 960)
    my_mission.setViewpoint(0)

    # Attempt to start a mission:
    max_retries = 3
    for retry in range(max_retries):
        try:
            agent_host.startMission( my_mission, my_client_pool, my_mission_record, 0, "Racer" )
            break
        except RuntimeError as e:
            if retry == max_retries - 1:
                print("Error starting mission:",e)
                exit(1)
                # sys.exit()
            else:
                time.sleep(2)

    # Loop until mission starts:
    print("\nWaiting for the mission to start on trial", iRepeat)
    world_state = agent_host.getWorldState()
    
    while not world_state.has_mission_begun:
        time.sleep(0.1)
        world_state = agent_host.getWorldState()
        for error in world_state.errors:
            print("Error:",error.text)
    
    # Loop until mission ends:
    print("Mission running...")
#     while world_state.is_mission_running:
#         time.sleep(0.1)
#         world_state = agent_host.getWorldState()

#         for error in world_state.errors:
#             print("Error:",error.text)
            
    # Check for actions and rewards here
    if (iRepeat + 1) % 5 == 0:
            print((iRepeat+1), 'Showing best policy:', end = " ")
            found_solution = racer.best_policy(agent_host)
            if found_solution:
                print('Found solution')
                print('Done')
                break
    else:
        print((iRepeat+1), 'Learning Q-Table:', end = " ")
        racer.run(agent_host)
    
    racer.clear_actions()  # clear list of actions for next run
    time.sleep(1)
    
    
    
    


print("\n\nMission ended.")
# Mission has ended.

ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Loading mission from xmls/testmap.txt
n = 10

Waiting for the mission to start on trial 0
Mission running...
1 Learning Q-Table: objs: {'log': 75, 'grass': 258, 'packed_ice': 105, 'Racer': (0.0, 0.0, -4.0)}
log, action completed: log
objs: {'log': 75, 'grass': 258, 'packed_ice': 105, 'Racer': (0.0, 0.0, -4.0)}
present_result, Reward: -10
